# NUEVOS DATOS DE NIGERIA

link al dataset: https://zenodo.org/record/1252141

In [1]:
import pandas as pd
import glob
import ssqueezepy #para convertir la señal en imagen
from ssqueezepy import cwt
from ssqueezepy.visuals import plot, imshow
import os
import mne #eeg analysis library
import scipy.io
#import torch.nn as nn
#import torch
import numpy as np
import re
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers, models

In [11]:
metadata = pd.read_csv('C:\\Users\\47575909\\Desktop\\metadata_nigeria.csv')
metadata

,subject.id,recordedPeriod,startTime,session.id,first_condition,remarks,Group,csv.file
0,6,270,26/9/2016 13:13,1,open,NaN,control,signal-6-1.csv.gz
1,9,271,26/9/2016 13:30,1,closed,NaN,control,signal-9-1.csv.gz
2,10,272,26/9/2016 13:36,1,open,eyes closed at 2:40,control,signal-10-1.csv.gz
3,11,274,26/9/2016 13:42,2,closed,no.11.1 failed >> 11.2 is the right one,control,signal-11-2.csv.gz
4,11,1,26/9/2016 13:42,1,closed,no.11.1 failed >> 11.2 is the right one,control,signal-11-1.csv.gz
...,...,...,...,...,...,...,...,...
220,634,268,1/10/2016 16:25,1,closed,NaN,epilepsy,signal-634-1.csv.gz
221,635,268,1/10/2016 16:31,1,open,"Every day a convulsion + snapt het niet goed, ...",epilepsy,signal-635-1.csv.gz
222,636,268,1/10/2016 16:36,1,closed,NaN,epilepsy,signal-636-1.csv.gz
223,637,268,1/10/2016 16:43,1,open,NaN,epilepsy,signal-637-1.csv.gz


In [12]:
#veo la mierda de los ids
groups = metadata.groupby('subject.id')

grp_keys = list(groups.groups.keys())
print(len(grp_keys))

222


In [21]:
metadata = metadata.drop([4, 10, 11, 64], axis=0)  # Elimina las filas que no me sirven
metadata.reset_index(drop=True, inplace=True)  # Restablece los índices y aplica los cambios

metadata
#SINO ELIMINA ESA PERSONA Y YA ESTA QCY

,subject.id,recordedPeriod,startTime,session.id,first_condition,remarks,Group,csv.file
0,6,270,26/9/2016 13:13,1,open,NaN,control,signal-6-1.csv.gz
1,9,271,26/9/2016 13:30,1,closed,NaN,control,signal-9-1.csv.gz
2,10,272,26/9/2016 13:36,1,open,eyes closed at 2:40,control,signal-10-1.csv.gz
3,11,274,26/9/2016 13:42,2,closed,no.11.1 failed >> 11.2 is the right one,control,signal-11-2.csv.gz
4,12,271,26/9/2016 13:47,1,open,NaN,control,signal-12-1.csv.gz
...,...,...,...,...,...,...,...,...
216,634,268,1/10/2016 16:25,1,closed,NaN,epilepsy,signal-634-1.csv.gz
217,635,268,1/10/2016 16:31,1,open,"Every day a convulsion + snapt het niet goed, ...",epilepsy,signal-635-1.csv.gz
218,636,268,1/10/2016 16:36,1,closed,NaN,epilepsy,signal-636-1.csv.gz
219,637,268,1/10/2016 16:43,1,open,NaN,epilepsy,signal-637-1.csv.gz


In [22]:
metadata = metadata.drop('startTime', axis=1)
metadata = metadata.drop('first_condition', axis=1)
metadata = metadata.drop('recordedPeriod', axis=1)
metadata = metadata.drop('remarks', axis=1)
metadata = metadata.drop('session.id', axis=1)

metadata['csv.file'] = metadata['csv.file'].str.replace('.csv.gz', '.csv')

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

print(metadata)

     subject.id     Group          csv.file
0             6   control    signal-6-1.csv
1             9   control    signal-9-1.csv
2            10   control   signal-10-1.csv
3            11   control   signal-11-2.csv
4            12   control   signal-12-1.csv
5            40   control   signal-40-1.csv
6            49   control   signal-49-1.csv
7            63   control   signal-63-1.csv
8            64   control   signal-64-1.csv
9            66   control   signal-66-1.csv
10           67   control   signal-67-1.csv
11           68   control   signal-68-1.csv
12           69   control   signal-69-1.csv
13           70   control   signal-70-1.csv
14           71   control   signal-71-1.csv
15           72   control   signal-72-1.csv
16           73   control   signal-73-1.csv
17           74   control   signal-74-1.csv
18           75   control   signal-75-1.csv
19           76   control   signal-76-1.csv
20           77   control   signal-77-1.csv
21           78   control   sign

In [6]:
'''dataset = pd.DataFrame()

for archivo_csv in os.listdir('C:\\Users\\47575909\\Desktop\\EEGs_Nigeria'):
    if archivo_csv.endswith('.csv'):
        archivo_csv = archivo_csv.replace(' ', '')
        
        # Obtener el nombre de archivo sin la extensión .csv
        archivo_base = archivo_csv.replace('.csv', '')

        # Verifica si hay filas en metadata que cumplen con la condición de búsqueda
        matching_rows = metadata.loc[metadata['csv.file'] == archivo_base]

        if not matching_rows.empty:  # Verifica si se encontraron coincidencias
            subject_id, label = matching_rows[['subject.id', 'Group']].values[0]

            # Leer el archivo CSV de EEG
            eeg_data = pd.read_csv(os.path.join(directorio_eeg, archivo_csv))

            # Agregar las columnas de subject ID y etiqueta al DataFrame de EEG
            eeg_data['subject.id'] = subject_id
            eeg_data['Group'] = label

            # Agregar los datos del EEG al conjunto de datos combinado
            dataset = pd.concat([dataset, eeg_data])

            print(f"Se encontró una coincidencia para {archivo_csv}: subject_id={subject_id}, Group={label}")
        else:
            print(f"No se encontraron coincidencias para {archivo_csv}")

# Puedes guardar el conjunto de datos combinado en un nuevo archivo CSV si lo deseas
dataset.to_csv('new_eeg_data.csv', index=False)'''

'dataset = pd.DataFrame()\n\nfor archivo_csv in os.listdir(\'C:\\Users\\47575909\\Desktop\\EEGs_Nigeria\'):\n    if archivo_csv.endswith(\'.csv\'):\n        archivo_csv = archivo_csv.replace(\' \', \'\')\n        \n        # Obtener el nombre de archivo sin la extensión .csv\n        archivo_base = archivo_csv.replace(\'.csv\', \'\')\n\n        # Verifica si hay filas en metadata que cumplen con la condición de búsqueda\n        matching_rows = metadata.loc[metadata[\'csv.file\'] == archivo_base]\n\n        if not matching_rows.empty:  # Verifica si se encontraron coincidencias\n            subject_id, label = matching_rows[[\'subject.id\', \'Group\']].values[0]\n\n            # Leer el archivo CSV de EEG\n            eeg_data = pd.read_csv(os.path.join(directorio_eeg, archivo_csv))\n\n            # Agregar las columnas de subject ID y etiqueta al DataFrame de EEG\n            eeg_data[\'subject.id\'] = subject_id\n            eeg_data[\'Group\'] = label\n\n            # Agregar los da

In [23]:
path = 'C:\\Users\\47575909\\Desktop\\EEGs_Nigeria'
all_files = glob.glob(os.path.join(path, "*.csv"))

def alfanumerico_key(item):
    partes = re.split(r'(\d+)', item)
    partes[1::2] = map(int, partes[1::2])
    return partes

all_files = sorted(all_files, key=alfanumerico_key)

def add_id_y(all_files, metadata):
    modified_dataframes_list = []

    for filename, subject_id, group in zip(all_files, metadata['subject.id'], metadata['Group']):
        df = pd.read_csv(filename, index_col=None, header=0)
        dataframe = df.iloc[:, 1:15].copy()
        # Obtén el número del archivo a partir del nombre
        file_number = int(re.search(r'(\d+)', os.path.basename(filename)).group(1))
        dataframe['subject_id'] = subject_id
        dataframe['y'] = group
        modified_dataframes_list.append(dataframe)

    return modified_dataframes_list

In [24]:
all_files

['C:\\Users\\47575909\\Desktop\\EEGs_Nigeria\\signal-6-1.csv',
 'C:\\Users\\47575909\\Desktop\\EEGs_Nigeria\\signal-9-1.csv',
 'C:\\Users\\47575909\\Desktop\\EEGs_Nigeria\\signal-10-1.csv',
 'C:\\Users\\47575909\\Desktop\\EEGs_Nigeria\\signal-11-2.csv',
 'C:\\Users\\47575909\\Desktop\\EEGs_Nigeria\\signal-12-1.csv',
 'C:\\Users\\47575909\\Desktop\\EEGs_Nigeria\\signal-40-1.csv',
 'C:\\Users\\47575909\\Desktop\\EEGs_Nigeria\\signal-49-1.csv',
 'C:\\Users\\47575909\\Desktop\\EEGs_Nigeria\\signal-63-1.csv',
 'C:\\Users\\47575909\\Desktop\\EEGs_Nigeria\\signal-64-1.csv',
 'C:\\Users\\47575909\\Desktop\\EEGs_Nigeria\\signal-66-1.csv',
 'C:\\Users\\47575909\\Desktop\\EEGs_Nigeria\\signal-67-1.csv',
 'C:\\Users\\47575909\\Desktop\\EEGs_Nigeria\\signal-68-1.csv',
 'C:\\Users\\47575909\\Desktop\\EEGs_Nigeria\\signal-70-1.csv',
 'C:\\Users\\47575909\\Desktop\\EEGs_Nigeria\\signal-71-1.csv',
 'C:\\Users\\47575909\\Desktop\\EEGs_Nigeria\\signal-72-1.csv',
 'C:\\Users\\47575909\\Desktop\\EEGs_Niger

In [25]:
modified_dataframes = add_id_y(all_files, metadata)

dataframes_list = []

for dataframe in modified_dataframes:
    dataframes_list.append(dataframe)

all_dataframes = pd.concat(dataframes_list, axis=0, ignore_index=True)
all_dataframes.to_csv('eeg_new_data.csv', index=False)

In [ ]:
all_dataframes

In [20]:
id_list = sorted(all_dataframes['subject_id'].unique())

print(id_list)

[6, 9, 10, 11, 12, 40, 49, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 112, 113, 114, 116, 120, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 500, 501, 502, 503, 504, 505, 506, 507, 508, 509, 510, 511, 512, 513, 514, 515, 516, 517, 518, 519, 520, 521, 522, 523, 524, 525, 526, 527, 528, 529, 530, 531, 532, 533, 534, 535, 536, 537, 538, 539, 540, 541, 542, 543, 544, 545, 546, 547, 548, 549, 550, 551, 552, 553, 554, 555, 556, 557, 558, 559, 560, 561, 562, 563, 564, 565, 566, 567, 568, 569, 570, 571, 572, 573, 574, 575, 576, 577, 580, 581, 582, 583, 584, 585, 586, 587, 588, 589, 590, 591, 592, 593, 594, 595, 596, 597, 598, 599, 600, 601, 602, 603, 604, 605, 606, 607, 608, 609, 610, 611, 612, 613, 614, 615, 616, 617, 618, 619, 620, 621, 622, 623, 624, 625,